In [1]:
!pip install fastai

In [2]:
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 168 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.0+cu101 which is incompatible.


In [3]:
# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.3 MB 750 kB/s 
     |████████████████████████████████| 144 kB 12.7 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 130 kB 22.1 MB/s 
     |████████████████████████████████| 49 kB 2.6 MB/s 
     |████████████████████████████████| 743 kB 21.1 MB/s 
     |████████████████████████████████| 721 kB 46.8 MB/s 
     |████████████████████████████████| 636 kB 47.2 MB/s 
     |████████████████████████████████| 112 kB 52.9 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210722-py3-none-any.whl size=60612 sha256=638ee7e345f93c90b9b05f3af6cb45f9993c0e8e91afdd8d8bde5d11a83a5df9
  Stored in directory: /root/.cache/pip/wheels/a6/99/bf/2ee4e2098402c3b918173613ff34384b0185e87198af9d44f9
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=4a17

In [4]:
# check pytorch installation: 
import torch, torchvision
assert torch.__version__.startswith("1.8")   # please manually install torch 1.8 if Colab changes its default version

In [5]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import os, json, cv2, random, io
from google.colab.patches import cv2_imshow
torch.cuda.current_device()
from fastai.vision import *
cuda = torch.device("cuda")
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import Boxes, BoxMode, PolygonMasks, RotatedBoxes
from detectron2.utils.file_io import PathManager
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from pycocotools.coco import COCO
from tqdm import tqdm_notebook
import urllib.request
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (20.0, 10.0)
import contextlib
import datetime
import logging
import pycocotools.mask as mask_util
from fvcore.common.timer import Timer
from iopath.common.file_io import file_lock
from PIL import Image
import concurrent.futures
import argparse
import pathlib

In [8]:
class image_detection():
    def downloads_COCO_dataSet(self):
        !wget http://images.cocodataset.org/zips/val2017.zip
        !unzip -o val2017.zip > /dev/null

        !wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
        !unzip -o annotations_trainval2017.zip > /dev/null

        coco_annotation_file_path = "annotations/instances_val2017.json"
        coco_annotation = COCO(annotation_file=coco_annotation_file_path)
        
    def get_dataSet_from_coco(self, name_json_file,name_images, name_dataset):
        register_coco_instances(name_dataset, {}, name_json_file , name_images)
        dataset_dicts = DatasetCatalog.get(name_dataset)
        return dataset_dicts
    
    def get_predictor(self):
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
        predictor = DefaultPredictor(cfg)
        return predictor, cfg

    def save_predict(self,cfg, dataset_dicts,number_image):
        name_image=0
        os.mkdir('panoptic_seg')
        for d in random.sample(dataset_dicts,number_image):  
            im = cv2.imread(d["file_name"])
            panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
            v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
            cv2.imwrite('panoptic_seg/'+str(name_image)+'.png',out.get_image()[:, :, ::-1])
            cv2.waitKey()
            name_image+=1

In [12]:
if __name__ == '__main__':
    # Create object of class detector_images
    obj_detector_images=image_detection()
    obj_detector_images.downloads_COCO_dataSet()
    name_dataset="dataset_coco3"
    dataset_dicts=obj_detector_images.get_dataSet_from_coco("annotations/instances_val2017.json","./val2017",name_dataset)
    predictor, cfg=obj_detector_images.get_predictor()
    obj_detector_images.save_predict(cfg,dataset_dicts,29)

--2021-07-24 13:36:21--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.193.153
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.193.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip.1’

val2017.zip.1       100%[===================>] 777.80M  36.9MB/s    in 22s     

2021-07-24 13:36:43 (35.5 MB/s) - ‘val2017.zip.1’ saved [815585330/815585330]

--2021-07-24 13:36:50--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.73.212
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.73.212|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

loading annotations into memory...
Done (t=0.86s)
creating index...
index create

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}
